# Ecuaciones Diferenciales Ordinarias con SciPy

## Ejemplo: Concentración de sal en un tanque

Encuentre la ecuación diferencial para la sal acumulada en untanque para cualquier tiempo. La concentración de sal es la masa de sal por unidad de volumen de la mezcla. Suponer que $c_{in}(t)$ es la razón a la que ingresa sal en el tanque y la razón a la que sale es $10c_{in}(t)$.

La ecuación derivada de la Ley de Balance es:

\begin{align}
\frac{dS}{dt} = 10 c_{in}(t) -\frac{1}{10} S(t)
\end{align}

La solución exacta es:
\begin{align}
S(t) = s_0 e^{\frac{-t}{10}} + 10e^{\frac{-t}{10}}\int_0^t c_{in}(s)e^{\frac{s}{10}} ds ,
\end{align}
donde $s_0$ es la condición inicial y $c_{in}(t)$ es una función del tiempo. 

Resolvamos el problema numericamente con $s_0 = 0.3$ and $c_{in}(t) = 0.1$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [ ]:
# t
t = np.linspace(0, 5, 200)

# condición inicial
S0 = 0.3

# Constant
cin = 0.1

In [ ]:
# definimos la funcion

def fn(S, t, cin):
    return 10*cin - S/10

In [ ]:
# Solución numérica

S_num = odeint(fn, S0, t, args=(cin,))

In [ ]:
# Solución analítica

S_exact = S0 * np.exp(-t/10) + 100*cin*(1-np.exp(-t/10))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 6))
axs[0].plot(t, S_num, label="solución numérica", c="black")
axs[1].plot(t, S_exact, label="solución exacta", c="black")
axs[0].set_title("solución numérica")
axs[1].set_title("solución exacta")
plt.show()

#### ¿Cuál es el efecto de la condición inicial?

#### Suponga funciones del tiempo para $c_{in}$ y grafique

En particular, considere $c_{in}(t)=0.2 + 0.1 sen t$.

## Ejercicio

Encuentre la ecuación diferencial para la concentración de un contaminante en un lago.

Suponga que $C(t)$ es la concentración del contaminante en el lago al tiempo $t$ y sea $F$ la razón a la que el agua fluye en el lago en $m^3/día$.

# Solución mediante cálculo simbólico

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import matplotlib as mpl
# mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.sans-serif'] = 'stix'

In [ ]:
import sympy
sympy.init_printing()

In [ ]:
from scipy import integrate

### Ejemplo: Ley de enfriamiento de Newton

La Ley de enfriamiento de Newton es:

$$\frac{dT(t)}{dt} = k \Big( T(t) - T_a \Big) $$

donde $T(t)$ es la temperatura de un cuerpo y $T_a$ es la temperatura del ambiente a su alrededor.
Para resolver esta ecuación con scipy procedemos como sigue:

In [ ]:
# Primero definimos los simbolos y representamos la función desconocida.
t, k, T0, Ta = sympy.symbols("t, k, T_0, T_a")
T = sympy.Function("T")

In [ ]:
# Definimos la ecuación a resolver:
ode = T(t).diff(t) + k*(T(t) - Ta)

In [ ]:
ode

In [ ]:
# Resolvemos para hallar la solución general:
ode_sol = sympy.dsolve(ode)

In [ ]:
ode_sol 

In [ ]:
# Se pueden referir los dos lados de la solución:
ode_sol.lhs

In [ ]:
ode_sol.rhs

In [ ]:
# Se proporcionan los valores iniciale smediante un diccionario clave-valor
ics = {T(0): T0}

In [ ]:
ics

In [ ]:
# Y resolver con condiciones iniciales:
C_eq = ode_sol.subs(t, 0).subs(ics)

In [ ]:
C_eq

In [ ]:
# Se resuelve la ecuación anterior para determinar la constante de integración
C_sol = sympy.solve(C_eq)

In [ ]:
C_sol

In [ ]:
ode_sol.subs(C_sol[0])

Se puede resolver para diferentes valores de los parámetros de manera concisa y graficar las soluciones.

In [ ]:
def apply_ics(sol, ics, x, known_params):

    free_params = sol.free_symbols - set(known_params)
    eqs = [(sol.lhs.diff(x, n) - sol.rhs.diff(x, n)).subs(x, 0).subs(ics)
           for n in range(len(ics))]
    sol_params = sympy.solve(eqs, free_params)
    return sol.subs(sol_params)

In [ ]:
ode_sol

In [ ]:
apply_ics(ode_sol, ics, t, [k, Ta])

In [ ]:
ode_sol = apply_ics(ode_sol, ics, t, [k, Ta]).simplify()

In [ ]:
ode_sol

In [ ]:
y_x = sympy.lambdify((t, k), ode_sol.rhs.subs({T0: 5, Ta: 1}), 'numpy')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))

x = np.linspace(0, 4, 100)

for k in [1, 2, 3]:
    ax.plot(x, y_x(x, k), label=r"$k=%d$" % k)

ax.set_title(r"$%s$" % sympy.latex(ode_sol), fontsize=18)
ax.set_xlabel(r"$x$", fontsize=18)
ax.set_ylabel(r"$y$", fontsize=18)
ax.legend()

fig.tight_layout()

### Ejercicio: Resolver simbólicamente la ecuación de la concentración de sal en el tanque y el modelo del lago, y graficas.